# **Task D:**

In [22]:
## Code to access the FashionMNIST
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from numpy import genfromtxt

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
root="data",
train=True,
download=True,
transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
root="data",
train=False,
download=True,
transform=ToTensor(),
)
print(training_data)
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [23]:
original_data = training_data
size_of_original_data = len(original_data)
print(size_of_original_data)


train_fraction = 0.70
val_fraction = 0.30

train_dataset_size = int(train_fraction * size_of_original_data)
val_dataset_size = int(val_fraction * size_of_original_data)

# Split whole original data into train, val and test datsets
train_dataset, val_dataset=  torch.utils.data.random_split(original_data,
                                                           [train_dataset_size,
                                                            val_dataset_size])
# Sanity checking
print(f" Train set Size: {len(train_dataset)}")
print(f" Val set Size: {len(val_dataset)}")
print(f" test set Size: {len(test_data)}")

60000
 Train set Size: 42000
 Val set Size: 18000
 test set Size: 10000


In [24]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
val_dataloader= DataLoader(dataset=val_dataset, batch_size=200, shuffle=True)
test_dataloader= DataLoader(dataset=test_data, batch_size=200, shuffle=True)
print(train_dataloader)
print(val_dataloader)
print(test_dataloader)

In [ ]:
# Simple way to define a Fully connected feed forward ANN
'''
model = nn.Sequential(
nn.Linear(28*28,64),
nn.ReLU(),
nn.Linear(64,64),
nn.ReLU(),
nn.Linear(64,10)
)


nn.Flatten()'''

'\nmodel = nn.Sequential(\nnn.Linear(28*28,64),\nnn.ReLU(),\nnn.Linear(64,64),\nnn.ReLU(),\nnn.Linear(64,10)\n)\n\n\nnn.Flatten()'

In [25]:
# fexible model:

class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    self.l1=nn.Linear(28*28,300)
    self.l2=nn.Linear(300,100)
    self.output=nn.Linear(100,10)
    

    self.relu=nn.ReLU()
    
  def forward(self,x):
    
    h1=self.relu(self.l1(x))
    #h2=nn.functional.relu(self.l2(h1))
    h2=self.relu(self.l2(h1))
    #h3 = self.relu(self.l2(h1))
    output=self.output(h2)
    #logits=self.l3(do)
    return output


In [26]:
model= ResNet()
optim = torch.optim.SGD(model.parameters(), lr=0.001)

In [27]:
cost_function = nn.CrossEntropyLoss()

In [14]:
model=ResNet()
optim = torch.optim.SGD(model.parameters(), lr=0.01)
cost_function = nn.CrossEntropyLoss()

training_minibatch_Js =[]
training_minibatch_Js_mean=[]
accuracy_list_1=[]

nr_epochs = 500    # Try
for epoch_i in range(nr_epochs):
  for X_batch, y_batch in train_dataloader:
    #print(X_batch)
    
    b= X_batch.size(0)
    X_batch=X_batch.view(b,-1)

    y_preds = model(X_batch)
    cost = cost_function(y_preds, y_batch)

    optim.zero_grad() # Set the grads of all model params to zero.

    #Forward pass
   

    cost.backward() # compute and populate gradients of model params
    
    optim.step() # instruct optimizer to take one update step (using SGD)
    
    training_minibatch_Js.append(cost.item())
    accuracy_list_1.append(y_batch.eq(y_preds.detach().argmax(dim=1)).float().mean())
    #print(f'epoch {epoch_i},train cost:{training_minibatch_Js}')
  print(f'epoch {epoch_i},train cost:{torch.tensor(training_minibatch_Js).mean():.4f}  &  train accuracies: {torch.tensor(accuracy_list_1).mean():.4f}')
  
  
  val_track=[]
  accuracy_list_2=[]
  for batch in val_dataloader:
    x, y = batch
    b= x.size(0)
    x=x.view(b,-1)
    with torch.no_grad():
      y_preds=model(x)
    cost= cost_function(y_preds, y)
    val_track.append(cost.item())
    accuracy_list_2.append(y.eq(y_preds.detach().argmax(dim=1)).float().mean())
  #print(f'Epoch {epoch_i},validation cost:{torch.tensor(val_track).mean():.4f}  &   validation accuracies: {torch.tensor(accuracy_list_2).mean():.4f}')

# Plot your cost
plt.figure(figsize=[10,5])
plt.plot(training_minibatch_Js)
plt.plot(val_track)
plt.xlabel(f"update step i on mini-batch")
plt.ylabel('Cost')
plt.title('Cost during training on train-set (per mini-batch)')
plt.grid()
plt.show()

epoch 0,train cost:2.3030  &  train accuracies: 0.0979
epoch 1,train cost:2.3008  &  train accuracies: 0.1183
epoch 2,train cost:2.2989  &  train accuracies: 0.1289
epoch 3,train cost:2.2969  &  train accuracies: 0.1460
epoch 4,train cost:2.2949  &  train accuracies: 0.1607
epoch 5,train cost:2.2926  &  train accuracies: 0.1818
epoch 6,train cost:2.2900  &  train accuracies: 0.1995
epoch 7,train cost:2.2870  &  train accuracies: 0.2187
epoch 8,train cost:2.2834  &  train accuracies: 0.2374
epoch 9,train cost:2.2790  &  train accuracies: 0.2554
epoch 10,train cost:2.2735  &  train accuracies: 0.2707
epoch 11,train cost:2.2664  &  train accuracies: 0.2840
epoch 12,train cost:2.2572  &  train accuracies: 0.2981
epoch 13,train cost:2.2450  &  train accuracies: 0.3090
epoch 14,train cost:2.2292  &  train accuracies: 0.3193
epoch 15,train cost:2.2094  &  train accuracies: 0.3284
epoch 16,train cost:2.1861  &  train accuracies: 0.3368
epoch 17,train cost:2.1602  &  train accuracies: 0.3446
ep

KeyboardInterrupt: ignored

In [28]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.metrics import accuracy_score


# spliting the dataset 
original_data = training_data
size_of_original_data = len(original_data)
print(f'Data set size: {size_of_original_data}')


train_fraction = 0.70
val_fraction = 0.30

train_dataset_size = int(train_fraction * size_of_original_data)
val_dataset_size = int(val_fraction * size_of_original_data)


# Split whole original data into train, val and test datsets
train_dataset, val_dataset=  torch.utils.data.random_split(original_data,
                                                           [train_dataset_size,
                                                            val_dataset_size])
# Sanity checking
print(f" Train set Size: {len(train_dataset)}")
print(f" Val set Size: {len(val_dataset)}")
print(f" test set Size: {len(test_data)}")



train_dataloader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
val_dataloader= DataLoader(dataset=val_dataset, batch_size=200, shuffle=True)
test_dataloader= DataLoader(dataset=test_data, batch_size=200, shuffle=True)


# fexible model:
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    self.l1=nn.Linear(28*28,300)
    self.l2=nn.Linear(300,100)
    self.output=nn.Linear(100,10)

    self.relu=nn.ReLU()
    self.sigmoid= nn.Sigmoid()
    
  def forward(self,x):
    h1=self.relu(self.l1(x))
    h2=self.relu(self.l2(h1))
    output=self.output(h2)
    return output





model=ResNet()
cost_function = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)

def evaluate_model_performance(dataset, model):
  d_loader = DataLoader(dataset = dataset, batch_size=len(dataset))
  cost_function=nn.CrossEntropyLoss()
  model.eval()

  with torch.no_grad():
    for x,y in d_loader:
      raw_y_preds=model(x) 
    y_class_preds= raw_y_preds.argmax(dim=1) 
    eval_cost= cost_function(raw_y_preds, y).item()
  model.train()
  eval_acc=accuracy_score(y_pred=y_class_preds,y_true=y)

  return eval_cost, eval_acc


training_minibatch_Js=[]
val_cost_l=[]
val_acc_l=[]
trian_cost_l=[]
train_acc_l=[]

nr_epochs=50
for epoch_i in range(nr_epochs):
  #if epoch_i%eval_every_kth ==0:
  model.eval()
  #print('Starting the frist epoc')
  train_cost, train_acc= evaluate_model_performance(model=model, dataset = train_dataset) # train cost and accuracy on train_dataset
  trian_cost_l.append(train_cost)
  train_acc_l.append(train_acc)
  print(f" In Epoch No. {nr_epochs}:  Cost= {val_cost:.4f} &  Accuracy: {train_acc:.4f}")



  val_cost, val_acc= evaluate_model_performance(model=model, dataset = val_dataset)# validation cost and accuracy on val_dataset
  val_cos_lt.append(val_cost)
  val_acc_l.append(val_acc)
  print(f"In Epoch No. {nr_epochs}  Cost: {train_cost:.4f} & Accuracy: {val_acc:.4f}")

  model.train()



   
    
  training_minibatch_Js=[] 
  for image, label in train_dataloader:
    label_pred = model(image)
    cost = cost_function(label_pred , label)
    optim.zero_grad() # Set the grads of all model params to zero.
    cost.backward() # compute and populate gradients of model params
    optim.step() # instruct optimizer to take one update step (using SGD)
    
    training_minibatch_Js.append(cost)
    print(f' in epoch {epoch_i} the last cost of epoch is {training_minibatch_Js[-1]}')


print("****************************")
print(f"final cost : {training_minibatch_Js[-1] }")

# EVALUATE Model 'performance' on whole train and Validation AND TEST dataset
# test_cost, test_acc= evaluate_model_performance(model=model, dataset = test_dataset)



# ploting ###################################################33##################################3333
# plotting cost 
plt.plot(training_minibatch_Js)
plt.xlabel(f"update step i on mini-batch")
plt.ylabel('Cost')
plt.title('Cost during training on train-set (per mini-batch)')
plt.grid()
plt.show()




# the average cross-entropy
plt.plot(trian_cost, label='Train Set', color='b')
plt.plot(val_cost, label='Validation Set', color='o')
plt.title('Eval: Cross_entropy cost throughout training')
plt.xlabel('epoches')
plt.ylabel('Cross_entropy Cost')
plt.legend()
plt.grid()
plt.show()

# Plot accuracy
plt.plot(train_acc, label='Train Set', color='b')
plt.plot(val_acc, label='Validation Set', color='o')
plt.title('Accuracy performance throughout training')
plt.xlabel('epoches')
plt.ylabel(' Accuracy')
plt.legend()
plt.grid()
plt.show()





Data set size: 60000
 Train set Size: 42000
 Val set Size: 18000
 test set Size: 10000


RuntimeError: ignored

In [ ]:
 # Print all parameters in your neural network model
for p in model.named_parameters():
  print(p)

('l1.weight', Parameter containing:
tensor([[-0.0195,  0.0015,  0.0030,  ..., -0.0120, -0.0301, -0.0336],
        [-0.0116, -0.0162,  0.0065,  ..., -0.0204, -0.0110, -0.0328],
        [-0.0181,  0.0030, -0.0338,  ..., -0.0348,  0.0010, -0.0149],
        ...,
        [ 0.0200, -0.0302,  0.0127,  ...,  0.0183, -0.0318,  0.0170],
        [-0.0305, -0.0266,  0.0303,  ..., -0.0042, -0.0240, -0.0286],
        [-0.0316, -0.0267,  0.0288,  ...,  0.0039,  0.0147, -0.0083]],
       requires_grad=True))
('l1.bias', Parameter containing:
tensor([ 0.1310, -0.0285, -0.0162,  0.0870,  0.1320,  0.2093,  0.0885,  0.0007,
        -0.0726,  0.1408,  0.1784,  0.0437, -0.1845,  0.1472,  0.1218,  0.0957,
         0.1591, -0.0370, -0.0106,  0.1124, -0.1303,  0.0709,  0.0710,  0.0973,
        -0.0430,  0.1865,  0.1030, -0.0508,  0.1641,  0.0191,  0.1128, -0.0211,
        -0.0989,  0.3755,  0.1963,  0.1184,  0.0555,  0.1088,  0.1571,  0.1467,
         0.1394,  0.0175,  0.0069, -0.0395,  0.0033,  0.0425,  0.017